In [1]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
import datetime

current_date = datetime.datetime.now().date()

target_date = datetime.date(2024, 6, 12)

if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [ ]:
import pandas as pd

df = pd.read_csv("data.csv")

In [ ]:
df.head()

# LLMChain

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [20]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [21]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe company that makes a product {product}"
)

In [22]:
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
product = "Queen Size Sheet Set"
chain.run(product=product)

# Simple chain sequence

In [27]:
from langchain.chains import SimpleSequentialChain

In [28]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe company that makes a product {product}"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [29]:
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 word description for the following company: {company_name}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [30]:
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two], verbose=True
)

In [ ]:
overall_simple_chain.run(product)

# Sequential chain

In [33]:
from langchain.chains import SequentialChain

In [34]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following text to english: {review}"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="english_review")

In [35]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence: {english_review}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

In [36]:
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review in: {review}"
)
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

In [37]:
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a followup response to the following summary in the specified language. Summary: {summary}, Language: {language}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup")

In [45]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["english_review", "summary", "language", "followup"],
    verbose=True,
)

In [ ]:
review = df.Review[5]
overall_chain(review)

# Router Chain

In [62]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [53]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template,
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template,
    },
]

In [54]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [55]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [56]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [57]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [58]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [59]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt)

In [ ]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [ ]:
chain.run("What is the speed of light?")

In [ ]:
chain.run("What is the capital of France?")

In [ ]:
chain.run("What is the derivative of x^2?")